## pandas11; pivot, pivot_table

In [2]:
import numpy as np
import pandas as pd

### pivot => 그룹으로 다시 나누지만 집계가 필요하지 않다
- 데이터 프레임에서 두개의 열을 이용하여=> 행과 열 인덱스로 reshpae 된 테이브을 통한 집계
- pivot(index,columns,data.values)
- index와 column의 조합으로 해당하는 값이 없거나 1개 존재해야 가능한거다
- 한가지 주의 사항은 index와 column의 조합으로 존재하지 않는 값이 있을수 있다 => 이때는 Nan값이 된다

In [15]:
data = {
    'city' : ['seoul','seoul','seoul','pusan','pusan','pusan','gwangju','gwangju'],
    'year' : [2015,2010,2005,2015,2020,2000,2015,2010],
    'pop' : [100,200,300,400,500,600,700,800],
    'region' : ['수도권','수도권','수도권','경상도','경상도','경상도','전라도','전라도']
}
pivot_df = pd.DataFrame(data)
display(pivot_df)

,city,year,pop,region
0,seoul,2015,100,수도권
1,seoul,2010,200,수도권
2,seoul,2005,300,수도권
3,pusan,2015,400,경상도
4,pusan,2020,500,경상도
5,pusan,2000,600,경상도
6,gwangju,2015,700,전라도
7,gwangju,2010,800,전라도


In [16]:
pivot_df.pivot(index='city',columns='year',values='pop')

year,2000,2005,2010,2015,2020
city,,,,,
gwangju,NaN,NaN,800.0,700.0,NaN
pusan,600.0,NaN,NaN,400.0,500.0
seoul,NaN,300.0,200.0,100.0,NaN


In [17]:
# 다중 인덱스도 가능하다
pivot_df.pivot(['region','city'],'year','pop')

,year,2000,2005,2010,2015,2020
region,city,,,,,
경상도,pusan,600.0,NaN,NaN,400.0,500.0
수도권,seoul,NaN,300.0,200.0,100.0,NaN
전라도,gwangju,NaN,NaN,800.0,700.0,NaN


In [18]:
import seaborn as sns
titanic_df = sns.load_dataset('titanic')
titanic_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [21]:
titanic_df.pivot('sex','class','survived') # error: Index contains duplicate entries, cannot reshape
# pivot_table을 사용해서 집계를 해야한다

ValueError: Index contains duplicate entries, cannot reshape

### pivot_table => 그룹으로 나누고 다시 집계까지
- pivot_table(data,values,index,columns,aggfunc) 추가적으로 margin margin_name 등등 있다

In [28]:
titanic_df.groupby(['sex','class'])[['survived']].mean().unstack()

survived                    
class      First    Second     Third
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447

In [33]:
titanic_df.pivot_table(index='sex',columns='class',values='survived') # aggfunc default:np.mean

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [35]:
titanic_df.pivot_table(index='sex',columns='class',values='survived',aggfunc=np.sum) 

class,First,Second,Third
sex,,,
female,91,70,72
male,45,17,47


In [39]:
# values를 반드시 줄 필요는 없다
# values 주지 않으면 모든 열의 가능한 조합에 대해 aggfunc 집계한다
titanic_df.pivot_table(index='sex',columns='class') 

adult_male                            age                        \
class       First    Second     Third      First     Second      Third   
sex                                                                      
female    0.00000  0.000000  0.000000  34.611765  28.722973  21.750000   
male      0.97541  0.916667  0.919308  41.281386  30.740707  26.507589   

           alone                            fare  ...     parch pclass         \
class      First    Second     Third       First  ...     Third  First Second   
sex                                               ...                           
female  0.361702  0.421053  0.416667  106.125798  ...  0.798611      1      2   
male    0.614754  0.666667  0.760807   67.226127  ...  0.224784      1      2   

                 sibsp                      survived                      
class  Third     First    Second     Third     First    Second     Third  
sex                                                                       
female     3  0.553191  0.486842  0.895833  0.968085  0.921053  0.500000  
male       3  0.311475  0.342593  0.498559  0.368852  0.157407  0.135447  

[2 rows x 24 columns]

In [40]:
# 성별과 객실등급에 따른 승객의 수를 집계한다면?
titanic_df.pivot_table(index='sex',columns='class',aggfunc='size')

class,First,Second,Third
sex,,,
female,94,76,144
male,122,108,347
